In [2]:
import sqlalchemy as db
from sqlalchemy import create_engine, inspect, MetaData
import pandas as pd

In [3]:
from sqlalchemy.sql import func, and_, desc, alias, text

In [4]:
pd.options.display.max_rows=200

In [5]:
engine = db.create_engine('sqlite:///C:\\Users\\peter\\Documents\\Projekte\\Zähler\\tc_all_new.db', echo = False)

In [6]:
conn = engine.connect()

In [9]:
metadata = db.MetaData(bind=engine)

TypeError: MetaData.__init__() got an unexpected keyword argument 'bind'

In [8]:
table = db.Table('tcdata_CH_0084_01', metadata, autoload = True)

TypeError: Additional arguments should be named <dialectname>_<argument>, got 'autoload'

#### Voila, mit der so definierten Tabelle kann ich die Abfragen starten!

In [8]:
q = db.select(table)
result = conn.execute(q)
row = result.fetchmany(4)
row

[('2021-07-27 19:53:00+02:00', 900.0, 68.9000015, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 19:58:00+02:00', 840.0, 65.5, 60.0, 73.0, 0.0, 0.0),
 ('2021-07-27 20:03:00+02:00', 840.0, 67.8000031, 60.0, 72.0, 0.0, 0.0),
 ('2021-07-27 20:08:00+02:00', 600.0, 79.3000031, 0.0, 0.0, 0.0, 0.0)]

#### Intro eines Alias für table (WICHTIG: *Alias* gilt nur für Tabellen. **Bemerkung:** Spalten kann man mit *Labels* umbenennen...

In [63]:
t = table.alias()

In [64]:
q = db.select(t.c.time, func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time).label('Stunde'), \
              func.avg(t.c.vfr11).label('AVG_flow'), func.avg(t.c.s12).label('AVG_speed'), func.count(t.c.time).label('Anzahl')). \
where(and_(t.c.s12 >=0, t.c.s12 <40 )) .\
group_by(func.strftime('%Y', t.c.time), func.strftime('%m', t.c.time), func.strftime('%d', t.c.time), func.strftime('%H', t.c.time)). \
having(func.count(t.c.time) > 50). \
order_by(func.strftime('%m', t.c.time).desc(), func.strftime('%d', t.c.time))
result = conn.execute(q)
row = result.fetchmany(10)
row

[('2021-12-01 17:00:00+01:00', '2021', '12', '01', '16', 1156.3636363636363, 18.894545495272723, 55),
 ('2021-12-02 16:02:00+01:00', '2021', '12', '02', '15', 1320.0, 26.607142805, 56),
 ('2021-12-02 17:01:00+01:00', '2021', '12', '02', '16', 1374.4444444444443, 23.35555557185185, 54),
 ('2021-12-08 16:00:00+01:00', '2021', '12', '08', '15', 1472.2222222222222, 30.675926038888882, 54),
 ('2021-12-08 17:00:00+01:00', '2021', '12', '08', '16', 1244.7272727272727, 22.232727330727275, 55),
 ('2021-11-11 17:00:00+01:00', '2021', '11', '11', '16', 1880.0, 31.350000138888888, 54),
 ('2021-11-15 17:00:00+01:00', '2021', '11', '15', '16', 1877.3076923076924, 30.96923076346154, 52),
 ('2021-11-16 17:00:00+01:00', '2021', '11', '16', '16', 1939.245283018868, 33.03018853962264, 53),
 ('2021-11-29 07:00:00+01:00', '2021', '11', '29', '06', 432.22222222222223, 6.764814816481482, 54),
 ('2021-10-01 17:00:00+02:00', '2021', '10', '01', '15', 1711.0, 28.153333509999996, 60)]

#### Und nun die (fast) gleiche Query unter Verwendung der Label bei HAVING, ORDER_BY und Rundung der Averages und **localtime** für den Wechsel von UTC auf CH-Time

In [77]:
q = db.select(t.c.time, func.count(t.c.time).label('Anzahl'), func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), \
               func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')). \
where(and_(t.c.s12 >0, t.c.s12 <40 )) .\
group_by(func.strftime('%Y', t.c.time), func.strftime('%m', t.c.time), func.strftime('%d', t.c.time), func.strftime('%H', t.c.time)). \
having(func.count(t.c.time) > 50). \
order_by(func.count('Anzahl').desc())
result = conn.execute(q)
row = result.fetchall()

In [78]:
print("Anzahl Std  mit Stau: ", len(row))

Anzahl Std  mit Stau:  0


In [79]:
df_stau= pd.read_sql_query(q, conn)
df_stau.head()

,time,Anzahl,Jahr,Monat,Tag,Stunde,AVG_flow,AVG_speed


#### Präp Loop, indem ich eine Liste mit allen Tabellen erstelle:

In [41]:
inspector = inspect(engine)

In [42]:
tab = inspector.get_table_names()
tab = tab[:-1]

In [ ]:
import os
os.getcwd()

'C:\\Users\\peter\\Coding\\Jupyter'

In [87]:
ListAnzahl = []
DfList = []
for table in tab:
    t = db.Table(table, metadata, autoload = True)
    ListAnzahl.append(table[10:])
    q = db.select(t.c.time, func.count(t.c.time).label('Anzahl'), func.strftime('%Y', t.c.time).label('Jahr'), func.strftime('%m', t.c.time).label('Monat'), \
              func.strftime('%d', t.c.time).label('Tag'), func.strftime('%H', t.c.time, 'localtime').label('Stunde'), \
               func.round(func.avg(t.c.vfr11)).label('AVG_flow'), func.round(func.avg(t.c.s12)).label('AVG_speed')). \
    where(and_(t.c.s12 >0, t.c.s12 <40 )) .\
    group_by(func.strftime('%Y', t.c.time), func.strftime('%m', t.c.time), func.strftime('%d', t.c.time), func.strftime('%H', t.c.time)). \
    having(func.count(t.c.time) > 50). \
    order_by(func.count('Anzahl').desc()) 
    result = conn.execute(q)
    row = result.fetchall()
    ListAnzahl.append(len(row)) 
    df = pd.read_sql_query(q, conn)
    df.to_csv(r'C:\Users\peter\Coding\Jupyter\Stau\s_' + table[10:] + '.csv', index = False)

In [76]:
print(ListAnzahl)

['0020_01', 0, '0020_02', 11, '0020_03', 8, '0020_04', 9, '0020_05', 83, '0020_06', 58, '0020_07', 62, '0020_08', 0, '0084_01', 153, '0084_02', 53, '0084_03', 0, '0084_04', 0, '0150_01', 0, '0150_02', 0, '0240_01', 0, '0240_02', 0, '0240_03', 74, '0240_04', 74, '0240_05', 2, '0240_06', 3, '0240_07', 1, '0240_08', 0, '0243_01', 0, '0243_02', 0, '0243_03', 4, '0243_04', 4, '0274_01', 2, '0274_02', 2, '0274_03', 2, '0274_04', 3, '0274_05', 5, '0274_06', 0, '0282_01', 5, '0282_02', 20, '0282_03', 24, '0282_04', 1, '0282_05', 1, '0282_06', 1, '0286_01', 0, '0286_02', 1, '0286_03', 1, '0286_04', 1, '0286_05', 0, '0286_06', 0, '0286_07', 0, '0286_08', 0, '0287_01', 0, '0287_02', 0, '0287_03', 0, '0287_04', 0, '0528_01', 0, '0528_02', 0, '0528_03', 0, '0528_04', 0, '0529_01', 0, '0529_02', 51, '0529_03', 113, '0529_04', 0, '0529_05', 0, '0601_01', 4, '0601_02', 1, '0601_03', 0, '0601_04', 44, '0602_01', 2, '0602_02', 1, '0602_03', 4, '0602_04', 9, '0605_01', 1, '0605_02', 0, '0605_03', 2, '060